In [1]:
import numpy as np
import pandas as pd
from chromatography import *
from separation_utility import *
from torch import optim, tensor
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mayavi import mlab
%matplotlib qt

In [2]:
alists = []
alists.append(pd.read_csv(f'../data/GilarSample.csv'))
alists.append(pd.read_csv(f'../data/Peterpeptides.csv'))
alists.append(pd.read_csv(f'../data/Roca.csv'))
alists.append(pd.read_csv(f'../data/Peter32.csv'))
alists.append(pd.read_csv(f'../data/Eosin.csv'))
alists.append(pd.read_csv(f'../data/Alizarin.csv'))
alists.append(pd.read_csv(f'../data/Controlmix2.csv'))


In [3]:
def loss_field(exp, taus, N = 200):
    phis = np.linspace(0, 1, N)
    losses = np.zeros((N, N))
    j = 0
    for phi1 in phis:
        i = 0
        for phi2 in phis:
            exp.reset()
            exp.run_all([phi1, phi2], taus)
            losses[i, j] = exp.loss()
            i += 1
        j += 1
    X, Y = np.meshgrid(phis, phis)
    
    return X, Y, losses

In [11]:
pol = PolicyGeneral(
            phi = nn.Sequential(
                PermEqui2_max(2, 4),
                nn.ELU(inplace=True),
                PermEqui2_max(4, 4),
                nn.ELU(inplace=True),
                PermEqui2_max(4, 4),
                nn.ELU(inplace=True),
            ),
            rho = Rho(3, 4, 4, .2, .01)
        )
losses, norms = reinforce_gen(
    alists = alists[1:], 
    policy = pol, 
    delta_taus = [.25, .25, 10], 
    num_episodes = 20000, 
    batch_size = 10, 
    lr = .01, 
    optim = lambda a, b: torch.optim.SGD(a, b),
    lr_decay_factor= 0.5,
    lr_milestones=1000,
    print_every = 100,
    baseline = .55,
    max_norm = None,
    max_rand_analytes = 30,
    min_rand_analytes = 10,
    rand_prob = 0.7
)


Loss: 1.014488621696717, epoch: 100/20000
Loss: 0.849440455911543, epoch: 200/20000
Loss: 0.6848969163195224, epoch: 300/20000
Loss: 0.7316335997110187, epoch: 400/20000
Loss: 0.6129049371616253, epoch: 500/20000
Loss: 0.8308539454868409, epoch: 600/20000
Loss: 0.7246419305239974, epoch: 700/20000
Loss: 0.6229216238747926, epoch: 800/20000
Loss: 0.5800404023891282, epoch: 900/20000
Loss: 0.6565654130757083, epoch: 1000/20000
Loss: 0.7117207351144262, epoch: 1100/20000
Loss: 0.8170514347806325, epoch: 1200/20000
Loss: 0.6979292489926776, epoch: 1300/20000
Loss: 0.6807244845327284, epoch: 1400/20000
Loss: 0.46496332055145706, epoch: 1500/20000
Loss: 0.8150214448476353, epoch: 1600/20000
Loss: 0.7468991985094526, epoch: 1700/20000
Loss: 0.662990404722896, epoch: 1800/20000
Loss: 0.941049760885875, epoch: 1900/20000
Loss: 0.7880074336674238, epoch: 2000/20000
Loss: 0.6050377775639899, epoch: 2100/20000
Loss: 0.71116734057086, epoch: 2200/20000
Loss: 0.7188661954956004, epoch: 2300/20000
Lo

Loss: 1.199653120228822, epoch: 18600/20000
Loss: 0.5974726220755688, epoch: 18700/20000
Loss: 0.936549494063264, epoch: 18800/20000
Loss: 0.6057334845385309, epoch: 18900/20000
Loss: 0.7861622154874432, epoch: 19000/20000
Loss: 0.7314900969052537, epoch: 19100/20000
Loss: 0.5536670588779234, epoch: 19200/20000
Loss: 1.037785575658305, epoch: 19300/20000
Loss: 0.6463716279150943, epoch: 19400/20000
Loss: 0.6972001645427612, epoch: 19500/20000
Loss: 0.6448050274571205, epoch: 19600/20000
Loss: 0.613626686368777, epoch: 19700/20000
Loss: 0.7040729641964649, epoch: 19800/20000
Loss: 0.8595781552648472, epoch: 19900/20000
Loss: 0.5585637961355732, epoch: 20000/20000


In [6]:
plt.plot(losses)
plt.show()

In [ ]:
plt.plot(mus[:, 0], label='Mu: phi1')
plt.plot(mus[:, 1], label='Mu: phi2')
#plt.plot(mus[:, 2], label='Mu: phi3')
#plt.plot(mus[:, 3], label='Mu: phi4')
plt.ylim((-0.1,1.1))
plt.legend()

In [ ]:
plt.plot(sigmas[:, 0], label='Sigma: phi1')
plt.plot(sigmas[:, 1], label='Sigma: phi2')
#plt.plot(sigmas[:, 2], label='Sigma: phi3')
#plt.plot(sigmas[:, 3], label='Sigma: phi4')
plt.ylim((-0.01,0.3))
plt.legend()

In [8]:
i = 0
exp = ExperimentAnalytes(k0 = alists[i].k0.values, S = alists[i].S.values, h=0.001,run_time=10.0)

In [10]:
import torch
mu, sig = pol(torch.Tensor(alists[0][['S', 'lnk0']].values))
exp.run_all(mu.detach().numpy(), [.25, .25, 10])

exp.print_analytes(title=f"Solvent Strength Program\nLoss:{round(exp.loss(), 4)}", rc=(10,10), angle=40)

In [ ]:
alists.append( pd.concat(alists, sort=True).sample(30))

In [ ]:
L = loss_field(exp, [.25, 10], 300)

In [ ]:
t = torch.randn((2, 3, 5))
a, b = t.max(0, keepdim=True)
t, a, b.shape